<a href="https://colab.research.google.com/github/raikovisk/ColorPicker/blob/main/Wan2.1_T2I_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content
!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI
!git clone https://github.com/city96/ComfyUI-GGUF /content/ComfyUI/custom_nodes/ComfyUI-GGUF
!pip install torchsde gguf

!apt install aria2 -qqy
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/city96/Wan2.1-T2V-14B-gguf/resolve/main/wan2.1-t2v-14b-Q3_K_S.gguf -d /content/ComfyUI/models/unet -o wan2.1-t2v-14b-Q3_K_M.gguf
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/city96/umt5-xxl-encoder-gguf/resolve/main/umt5-xxl-encoder-Q3_K_S.gguf -d /content/ComfyUI/models/clip -o umt5-xxl-encoder-Q3_K_M.gguf
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/Wan_2.1_ComfyUI_repackaged/resolve/main/split_files/vae/wan_2.1_vae.safetensors -d /content/ComfyUI/models/vae -o/wan_2.1_vae.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/vrgamedevgirl84/Wan14BT2VFusioniX/resolve/main/FusionX_LoRa/Wan2.1_T2V_14B_FusionX_LoRA.safetensors -d /content/ComfyUI/models/loras/FusionX -o Wan2.1_T2V_14B_FusionX_LoRA.safetensors

/content
Cloning into '/content/ComfyUI'...
remote: Enumerating objects: 29144, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 29144 (delta 33), reused 26 (delta 26), pack-reused 29098 (from 3)
Receiving objects: 100% (29144/29144), 75.39 MiB | 21.99 MiB/s, done.
Resolving deltas: 100% (19783/19783), done.
Cloning into '/content/ComfyUI/custom_nodes/ComfyUI-GGUF'...
remote: Enumerating objects: 783, done.
remote: Counting objects: 100% (484/484), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 783 (delta 439), reused 302 (delta 302), pack-reused 299 (from 1)
Receiving objects: 100% (783/783), 182.11 KiB | 8.28 MiB/s, done.
Resolving deltas: 100% (524/524), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 9.5 MB/s eta 0:00:00
The following additional packages will be installed:
  libaria2-0 libc-ares2
The f

In [ ]:
%cd /content/ComfyUI

import torch
import random, time
from PIL import Image
import numpy as np

from nodes import NODE_CLASS_MAPPINGS, load_custom_node
from comfy_extras import nodes_hunyuan, nodes_model_advanced

load_custom_node("/content/ComfyUI/custom_nodes/ComfyUI-GGUF")

UnetLoaderGGUF = NODE_CLASS_MAPPINGS["UnetLoaderGGUF"]()
CLIPLoaderGGUF = NODE_CLASS_MAPPINGS["CLIPLoaderGGUF"]()
LoraLoaderModelOnly = NODE_CLASS_MAPPINGS["LoraLoaderModelOnly"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()

CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
EmptyHunyuanLatentVideo = nodes_hunyuan.NODE_CLASS_MAPPINGS["EmptyHunyuanLatentVideo"]()

KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
ModelSamplingSD3 = nodes_model_advanced.NODE_CLASS_MAPPINGS["ModelSamplingSD3"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()

with torch.inference_mode():
    unet = UnetLoaderGGUF.load_unet("wan2.1-t2v-14b-Q3_K_M.gguf")[0]
    clip = CLIPLoaderGGUF.load_clip("umt5-xxl-encoder-Q3_K_M.gguf", "wan")[0]
    lora = LoraLoaderModelOnly.load_lora_model_only(unet, "FusionX/Wan2.1_T2V_14B_FusionX_LoRA.safetensors", 1.0)[0]
    vae = VAELoader.load_vae("wan_2.1_vae.safetensors")[0]

In [ ]:
with torch.inference_mode():
    seed = 0
    steps = 10
    cfg = 1.0
    sampler_name = "euler"
    scheduler = "beta"
    width = 1280
    height = 720
    length = 1
    positive_prompt = "In a dim, candlelit medieval hall, a queen walks slowly along a corridor lined with flickering torches. The camera tracks her from behind in a long, steady tracking shot, capturing the heavy drape of her velvet gown and the echo of her footsteps. Shadows dance across the stone walls, and dust swirls in the warm volumetric light. The mood is regal, tense, and quietly suspenseful."
    negative_prompt = "色调艳丽，过曝，静态，细节模糊不清，字幕，风格，作品，画作，画面，静止，整体发灰，最差质量，低质量，JPEG压缩残留，丑陋的，残缺的，多余的手指，画得不好的手部，画得不好的脸部，畸形的，毁容的，形态畸形的肢体，手指融合，静止不动的画面，杂乱的背景，三条腿，背景人很多，倒着走"
    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    model = ModelSamplingSD3.patch(lora, 1.0)[0]
    latent_image = EmptyHunyuanLatentVideo.generate(width, height, length)[0]
    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)
    samples = KSampler.sample(model, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image)[0]

    decoded = VAEDecode.decode(vae, samples)[0].detach()
    image = Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"/content/test.png")

Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0])